### Debug Collector Error

We have have issues which come from being unable to collect data from a url. This notebook gives some quickly configurable code with the problematic url  to replicate the error and also un a simple request to check if it's because of our user agent.

In [16]:
%load_ext autoreload
%autoreload 2

from digital_land.collect import Collector
from pathlib import Path
import requests

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
endpoint_url = "https://www.tendringdc.gov.uk/sites/default/files/documents/planning/CAD%20csv.csv"
plugin = ''

data_dir = Path('./data/debug_collector_error')
data_dir.mkdir(parents=True, exist_ok=True)

In [ ]:

collection_dir = data_dir / 'collection'
collection_dir.mkdir(parents=True, exist_ok=True)

collector = Collector(collection_dir=collection_dir)
collector.fetch(
        endpoint_url,
        plugin="",
        refill_todays_logs=True,
    )

url being fetched is: https://www.tendringdc.gov.uk/sites/default/files/documents/planning/CAD%20csv.csv


<FetchStatus.FAILED: 5>

If an error is appearing above then use the below code to use head requests to check for common causes which are:

* User Agent Error - this is when the request is only okay when we don't incldue our user agent. This is common as often web hosting providers stop rer-occuring user agents
* SSL Error - this happens when we can't verify the SSL certificate

Fo both these errors action needs to be take on the poviders end as we don't have control over these problems. if it's neither of these errors then it tends to imply that our code is adding complexity that is breaking it and you can review our code

In [15]:
try:
    # Default is verify=True, but you can be explicit
    response_original = requests.head(
        endpoint_url, 
        timeout=5, 
        verify=True, 
        headers={"User-Agent": collector.user_agent},
        allow_redirects=True
    )
    if response_original.status_code == 200:
        print(f"{endpoint_url} is OK (200). SSL verified.")
    else:
        response_no_user_agent = requests.head(endpoint_url, timeout=5, verify=True,allow_redirects=True )

        print(f"{endpoint_url} returned status {response_no_user_agent.status_code}. SSL verified.")
except requests.exceptions.SSLError as ssl_err:
    print(f"SSL verification failed for {endpoint_url}: {ssl_err}")
    try:
        response_no_verify = requests.head(
            endpoint_url, 
            timeout=5, 
            verify=False, 
            headers={"User-Agent": collector.user_agent},
            allow_redirects=True
        )
        if response_no_verify.status_code == 200:
            print(f"{endpoint_url} is OK (200) when SSL verification is disabled.")
        else:
            print(f"{endpoint_url} returned status {response_no_verify.status_code} when SSL verification is disabled.")
    except requests.RequestException as e:
        print(f"Error checking {endpoint_url} with SSL verification disabled: {e}")
except requests.RequestException as e:
    print(f"Error checking {endpoint_url}: {e}")


SSL verification failed for https://www.tendringdc.gov.uk/sites/default/files/documents/planning/CAD%20csv.csv: HTTPSConnectionPool(host='legacy.tendringdc.gov.uk', port=443): Max retries exceeded with url: /sites/default/files/documents/planning/CAD%20csv.csv (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1147)')))


/Users/oweneveleigh/repos4/digital-land-python/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.tendringdc.gov.uk'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


https://www.tendringdc.gov.uk/sites/default/files/documents/planning/CAD%20csv.csv is OK (200) when SSL verification is disabled.


/Users/oweneveleigh/repos4/digital-land-python/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'legacy.tendringdc.gov.uk'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
